# Chapter 3
## Loops and Iterators

# Enumerate
It's better than len + range!

In [ ]:
# Enumerate makes range(len(x)) pointless

for i, thing in enumerate(["a", "b", "c"]):
    print(i, thing)

In [ ]:
# Enumerate works by creating a generator that yields the enumerated item with an integer index

enumerated = enumerate(["a", "b", "c"])
next(enumerated)

In [ ]:
for i, thing in enumerated: # Discuss
    print(i, thing)


In [ ]:
# What will this do?
next(enumerated)

Aside: generator; what the flip?! DISCUSS

## Make use of `zip`

In [ ]:
from faker import Faker
fake = Faker()

# DID YOU KNOW? List comprehensions edition
# https://wiki.haskell.org/List_comprehension

some_names = [fake.name() for _ in range(9)]
char_counts = [len(n) for n in some_names]

Cool cool. But I want to access data from BOTH lists at the same time, now what?!

In [ ]:
longest_name = None
biggest_count = 0

for i in range(len(some_names)):
    count = char_counts[i]
    if count > biggest_count:
        longest_name = some_names[i]
        biggest_count = count

print(longest_name)

Hideous!!!

Surely, there's a better way Mr. Slatkin?!

In [ ]:
# Generators again?!
longest_name = None
biggest_count = 0

for name, count in zip(some_names, char_counts):
    if count > biggest_count:
        longest_name = name
        biggest_count = count

print(longest_name)

Wow! Thanks Mr. Slatkin! I'm sure there's no caveats right? Like if the lists aren't the same size?

In [ ]:
some_names.append("Video Center Rulez")
for name, count in zip(some_names, char_counts):
    print(name, count)

Damn! So `zip` will only yield a tuple until one of the wrapped sequences is exhausted. Well, good thing there's `strict` if I really care about truncation (or the cool itertools lib)

In [ ]:
try:
    for name, count in zip(some_names, char_counts, strict=True):
        print(name, count)
except ValueError:
    print("No way man!")

In [ ]:
from itertools import zip_longest

for name, count in zip_longest(some_names, char_counts, fillvalue=10000):
    print(name, count)

### Things to Remember
- zip can be used to iterate over multiple iterators in parallel
- zip makes a lazy generator that pops out tuples
- zip (by default) silently truncates output to the length of the shorter iterator
    - use strict to avoid this

# Avoid Else Blocks After for and while Loops

Yes, this _is_ a thing you can do in Python and _yes_ it _is_ confusing.

In [ ]:
for name in some_names[:2]:
    print(name)
else:
    print("Done!")

Why did the else execute? Shouldn't it have... done nothing? The list wasn't empty!!! Why doesn't this work like try/except/else?!

In [ ]:
for x in []:
    print("Unreachable code baby!")
else:
    print("And yet here we are.")

In theory, the `else` is sort of meant to be combined with `break`, in a search operation

In [ ]:
for name in some_names:
    if name == "Video Center Rulez":
        print(f"We found {name}!")
        break
else:
    print("No results found!")

Still, kind of confusing no? Could instead just use a result variable:

In [ ]:
def find_name(names, target) -> bool:
    found_it = False
    for name in names:
        if name == target:
            found_it = True
            break

    return found_it

find_name(some_names, "Video Center Rulez")

### Things to Remember
- the else block on for is goofy, probably just avoid using it

# Never Use for Loop Variables After the Loop Ends
Just because the loop ends, doesn't mean its var goes out of scope...

In [ ]:
for i in range(10):
    print(i)
print(i)

It's tempting to use this in _clever_ ways, but really, just avoid it, because you can't always be sure it will be there:

In [ ]:
for x in []:
    print(x)
# print(x)

In [ ]:
# But it gets worse!

for i in []:
    print(i)
print(i)

Whoops! That's kinda confusing! Yeah just.. don't do this. This leakage behavior is an artifact of how Python builds it's syntax tree. Note how this behavior does _not_ happen on list and generator comprehensions.